In [1]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from envs.rodent import RodentMultiClipTracking, RodentTracking
from preprocessing import mjx_preprocess as mjxp

from brax.training.agents.ppo import networks as ppo_networks

import numpy as np
import mediapy as media
import jax.numpy as jp
import mujoco
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from brax import envs
from preprocessing.mjx_preprocess import process_clip_to_train

2024-07-11 01:28:02.112437: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.82). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
reference_clip = process_clip_to_train('clips/all_snips.p')

/root/anaconda3/envs/vnl/lib/python3.11/site-packages/jax/_src/interpreters/xla.py:155: RuntimeWarning: overflow encountered in cast
  return np.asarray(x, dtypes.canonicalize_dtype(x.dtype))


In [3]:
envs.register_environment("rodent", RodentMultiClipTracking)

# Define the environment arguments
env_args = {
    "reference_clip": reference_clip,
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R"
    ],
    "appendage_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
        "skull"
    ],
    "walker_body_names": [
        "torso",
        "pelvis",
        "upper_leg_L",
        "lower_leg_L",
        "foot_L",
        "upper_leg_R",
        "lower_leg_R",
        "foot_R",
        "skull",
        "jaw",
        "scapula_L",
        "upper_arm_L",
        "lower_arm_L",
        "finger_L",
        "scapula_R",
        "upper_arm_R",
        "lower_arm_R",
        "finger_R"
    ],
    "joint_names": [
        "vertebra_1_extend",
        "hip_L_supinate",
        "hip_L_abduct",
        "hip_L_extend",
        "knee_L",
        "ankle_L",
        "toe_L",
        "hip_R_supinate",
        "hip_R_abduct",
        "hip_R_extend",
        "knee_R",
        "ankle_R",
        "toe_R",
        "vertebra_C11_extend",
        "vertebra_cervical_1_bend",
        "vertebra_axis_twist",
        "atlas",
        "mandible",
        "scapula_L_supinate",
        "scapula_L_abduct",
        "scapula_L_extend",
        "shoulder_L",
        "shoulder_sup_L",
        "elbow_L",
        "wrist_L",
        "scapula_R_supinate",
        "scapula_R_abduct",
        "scapula_R_extend",
        "shoulder_R",
        "shoulder_sup_R",
        "elbow_R",
        "wrist_R",
        "finger_R"
    ],
    "center_of_mass": "torso",
    "mjcf_path": "./assets/rodent.xml",
    "scale_factor": 0.9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "healthy_z_range": (0.2, 1.0),
    "reset_noise_scale": 0.1,
    "clip_length": 250,
    "sub_clip_length": 10,
    "ref_traj_length": 5,
    "termination_threshold": 5,
    "body_error_multiplier": 1.0,
    "min_steps": 10,
    "random_start":True,
}

env = envs.get_environment('rodent', **env_args)

## Checking reset & step functionality

**reset function does support `vmap` for parralel env processing, but not the step function**

In [4]:
key = random.PRNGKey(100)
jax.random.randint(key, (env._num_clips,), 0, 235)

Array([ 28, 118,  71,  54,   3, 182,  27, 200, 106, 140, 109, 113,  12,
       213, 186, 203, 164, 139, 189, 132,  42,  68, 206,  54,  60, 138,
        80, 213,  75, 150,  57, 134,   1, 141, 117,  49, 207, 196, 148,
        97, 203, 205,  99, 170, 113,  99, 191,  44, 107, 163,  81,  30,
        74, 127, 144,  97, 191, 174,  33, 157, 176,  86, 172, 123, 120,
        53,  80,  29, 113, 122,  24, 135,   2,  85, 152,  48, 150,  67,
       210,  24, 148,   3,  50, 149,  47, 215, 218, 197,  94,  10, 106,
       112, 185, 111,   2, 197,  77, 203,  10, 230, 106,  82, 229,  57,
         9, 190,  68, 186, 220, 180, 212, 201, 127, 200,  27,  51, 153,
       179,  45, 191, 213,   0, 176,  55, 170, 112,  23, 112,   4,  68,
        35, 137, 115, 212, 132, 196,  47, 101, 136, 134, 162, 127, 195,
       122, 117, 208,  88,  59, 179,  19,  28,  72, 112,  17,  53, 108,
       116, 220, 115,  96, 142, 231,  15,  68, 124,  23,  80, 219, 107,
       116,  55, 231, 139, 109, 160, 113,  57,  49, 105,  52,  2

In [5]:
key = random.PRNGKey(100)
reset_fn = jax.jit(env.reset)
step_fn = jax.jit(env.step)

In [6]:
# n_envs = 5
_, key_envs= jax.random.split(key)
env_state = reset_fn(key_envs)

Inititialize the environment with reset called first time

In [7]:
env_state.info

{'cur_clip_id': Array(499, dtype=int32),
 'cur_frame': Array(241, dtype=int32),
 'sub_clip_frame': Array(0, dtype=int32, weak_type=True),
 'termination_error': Array(-0.5599636, dtype=float32),
 'traj': Array([-0.20617531,  0.09366457,  0.06282681, -0.22510804,  0.08976174,
         0.06202527, -0.2122403 ,  0.05918474,  0.02767727, -0.2122403 ,
         0.05918474,  0.02767727, -0.2122403 ,  0.05918474,  0.02767727,
        -0.20586842,  0.09413065,  0.06297362, -0.22472633,  0.08995721,
         0.0623534 , -0.21207331,  0.05739938,  0.02868471, -0.21207331,
         0.05739938,  0.02868471, -0.21207331,  0.05739938,  0.02868471,
        -0.20546675,  0.09449291,  0.06335069, -0.22439913,  0.0900441 ,
         0.06291123, -0.2117454 ,  0.05687447,  0.02950223, -0.2117454 ,
         0.05687447,  0.02950223, -0.2117454 ,  0.05687447,  0.02950223,
        -0.2054224 ,  0.09443736,  0.06350307, -0.22460906,  0.09005074,
         0.06321929, -0.21223037,  0.05709274,  0.02985486, -0.21223

Inititialize the environment with reset called second time

In [8]:
_, new_key_env = jax.random.split(key_envs)
env_state = reset_fn(new_key_env)

In [9]:
env_state.info

{'cur_clip_id': Array(738, dtype=int32),
 'cur_frame': Array(108, dtype=int32),
 'sub_clip_frame': Array(0, dtype=int32, weak_type=True),
 'termination_error': Array(-1.1493816, dtype=float32),
 'traj': Array([-0.16082662,  0.14971048,  0.10406622, -0.17315409,  0.13138755,
         0.10413112, -0.15152259,  0.12799649,  0.06473141, -0.15152259,
         0.12799649,  0.06473141, -0.15152259,  0.12799649,  0.06473141,
        -0.16100302,  0.14942591,  0.10313655, -0.17316258,  0.13146493,
         0.10305461, -0.15096259,  0.12778363,  0.06369471, -0.15096259,
         0.12778363,  0.06369471, -0.15096259,  0.12778363,  0.06369471,
        -0.1609548 ,  0.14878199,  0.10276634, -0.17297348,  0.13145363,
         0.10245484, -0.15024397,  0.12769893,  0.0631542 , -0.15024397,
         0.12769893,  0.0631542 , -0.15024397,  0.12769893,  0.0631542 ,
        -0.16127068,  0.14832388,  0.10224934, -0.17297032,  0.13145158,
         0.10219319, -0.14975944,  0.12775725,  0.06302866, -0.14975

stepping the environment with the step function

In [10]:
step_fn = jax.jit(env.step)

In [11]:
random.normal(key, shape=(env.sys.nu,)).shape

(30,)

In [12]:
mu = 0
sigma = 0.3
action =  mu + sigma * random.normal(key, shape=(env.sys.nu,))
state = env_state.pipeline_state
# env.pipeline_step(state, action)
state.q.shape

(74,)

In [13]:
new_state = step_fn(env_state, action)

In [15]:
new_state.info

{'cur_clip_id': Array(738, dtype=int32),
 'cur_frame': Array(109, dtype=int32),
 'sub_clip_frame': Array(1, dtype=int32, weak_type=True),
 'termination_error': Array(-0.01149382, dtype=float32),
 'traj': Array([-1.61003023e-01,  1.49425909e-01,  1.03136554e-01, -1.73162580e-01,
         1.31464928e-01,  1.03054613e-01, -1.50962591e-01,  1.27783626e-01,
         6.36947080e-02, -1.50962591e-01,  1.27783626e-01,  6.36947080e-02,
        -1.50962591e-01,  1.27783626e-01,  6.36947080e-02, -1.60954803e-01,
         1.48781985e-01,  1.02766335e-01, -1.72973484e-01,  1.31453633e-01,
         1.02454841e-01, -1.50243968e-01,  1.27698928e-01,  6.31541982e-02,
        -1.50243968e-01,  1.27698928e-01,  6.31541982e-02, -1.50243968e-01,
         1.27698928e-01,  6.31541982e-02, -1.61270678e-01,  1.48323879e-01,
         1.02249339e-01, -1.72970325e-01,  1.31451577e-01,  1.02193192e-01,
        -1.49759442e-01,  1.27757251e-01,  6.30286559e-02, -1.49759442e-01,
         1.27757251e-01,  6.30286559e

stepping again

In [17]:
new_state = step_fn(new_state, action)
new_state.info

{'cur_clip_id': Array(738, dtype=int32),
 'cur_frame': Array(111, dtype=int32),
 'sub_clip_frame': Array(3, dtype=int32, weak_type=True),
 'termination_error': Array(-0.02996845, dtype=float32),
 'traj': Array([-1.61270678e-01,  1.48323879e-01,  1.02249339e-01, -1.72970325e-01,
         1.31451577e-01,  1.02193192e-01, -1.49759442e-01,  1.27757251e-01,
         6.30286559e-02, -1.49759442e-01,  1.27757251e-01,  6.30286559e-02,
        -1.49759442e-01,  1.27757251e-01,  6.30286559e-02, -1.61693200e-01,
         1.47677273e-01,  1.01653561e-01, -1.73141703e-01,  1.31432191e-01,
         1.01790078e-01, -1.49743766e-01,  1.27371818e-01,  6.26954436e-02,
        -1.49743766e-01,  1.27371818e-01,  6.26954436e-02, -1.49743766e-01,
         1.27371818e-01,  6.26954436e-02, -1.61682144e-01,  1.47180125e-01,
         1.01368077e-01, -1.73007488e-01,  1.31488383e-01,  1.01449654e-01,
        -1.49364397e-01,  1.26723528e-01,  6.29757419e-02, -1.49364397e-01,
         1.26723528e-01,  6.29757419e

## `vmap` reset function for parralel environments

In [18]:
reset_fn = jax.jit(jax.vmap(env.reset))
step_fn = jax.jit(jax.vmap(env.step))

n_envs = 10
key_envs = jax.random.split(key, n_envs)
env_state = reset_fn(key_envs)

env_state.info

{'cur_clip_id': Array([485, 401, 632,  71, 651, 258, 284, 312, 366, 194], dtype=int32),
 'cur_frame': Array([235, 187, 136, 149, 123,  10, 210,  94, 146, 112], dtype=int32),
 'sub_clip_frame': Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32, weak_type=True),
 'termination_error': Array([-0.56457305,  0.09398538, -1.6482213 , -0.5820372 , -0.8586699 ,
        -1.703099  , -0.81337774, -0.9759407 , -1.0845871 , -1.3452129 ],      dtype=float32),
 'traj': Array([[-0.15427066,  0.01494329,  0.05672465, ..., -0.06212887,
         -0.09165404, -0.09165404],
        [ 0.02545187, -0.09089029,  0.05754928, ..., -0.01286346,
         -0.09104721, -0.09104721],
        [-0.18914463,  0.07429746,  0.05606084, ...,  0.03004456,
         -0.03742177, -0.03742177],
        ...,
        [ 0.06721927,  0.00763874,  0.08064112, ...,  0.02001724,
          0.01832652,  0.01832652],
        [ 0.21447162, -0.10602634,  0.17612511, ..., -0.2219195 ,
          0.01180918,  0.01180918],
        [-0.2264196

In [19]:
new_key_env = jax.random.split(key_envs[0], n_envs)
env_state = reset_fn(new_key_env)
env_state.info

{'cur_clip_id': Array([467,  73, 426, 756, 356, 688, 409, 791, 404, 261], dtype=int32),
 'cur_frame': Array([169,  81,  84,  16,  22, 220,  69, 197, 142, 211], dtype=int32),
 'sub_clip_frame': Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32, weak_type=True),
 'termination_error': Array([-1.0384603 , -1.3318532 , -0.68277967, -2.1269138 , -1.3225667 ,
        -0.68879795, -0.48999393, -0.89248884, -0.90788174, -0.7953942 ],      dtype=float32),
 'traj': Array([[-0.20004347,  0.2385041 ,  0.06029586, ..., -0.08812524,
          0.01174304,  0.01174304],
        [-0.15528704,  0.00930448,  0.11902401, ..., -0.06174397,
          0.01036848,  0.01036848],
        [ 0.08066069,  0.04199532,  0.05919814, ..., -0.15245038,
         -0.18402584, -0.18402584],
        ...,
        [-0.09740867,  0.3776508 ,  0.05755525, ..., -0.12594199,
          0.00241748,  0.00241748],
        [ 0.0057214 ,  0.43774807,  0.05910759, ..., -0.07318569,
          0.00125737,  0.00125737],
        [ 0.3758247

In [22]:
mu = 0
sigma = 0.3
action =  mu + sigma * random.normal(key, shape=(n_envs, env.sys.nu))
action.shape

(10, 30)

In [23]:
new_state = step_fn(env_state, action)
new_state.info

{'cur_clip_id': Array([467,  73, 426, 756, 356, 688, 409, 791, 404, 261], dtype=int32),
 'cur_frame': Array([170,  82,  85,  17,  23, 221,  70, 198, 143, 212], dtype=int32),
 'sub_clip_frame': Array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32, weak_type=True),
 'termination_error': Array([-0.0103846 , -0.01331853, -0.0068278 , -0.02126914, -0.01322567,
        -0.00688798, -0.00489994, -0.00892489, -0.00907882, -0.00795394],      dtype=float32),
 'traj': Array([[-0.20036991,  0.23884232,  0.06018402, ..., -0.08873986,
          0.01693399,  0.01693399],
        [-0.15533067,  0.00956762,  0.11930922, ..., -0.06313196,
          0.00941299,  0.00941299],
        [ 0.08127106,  0.05134097,  0.06068143, ..., -0.3674417 ,
          0.16125631,  0.16125631],
        ...,
        [-0.09816636,  0.37509644,  0.05811431, ..., -0.16890828,
         -0.04522393, -0.04522393],
        [ 0.01007128,  0.4416503 ,  0.0586198 , ..., -0.11738526,
          0.00072166,  0.00072166],
        [ 0.3767978

step successfully

## Reference to SingleClip

In [ ]:
envs.register_environment("rodent_single", RodentTracking)

# Define the environment arguments
env_args = {
    "reference_clip": reference_clip,
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R"
    ],
    "appendage_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
        "skull"
    ],
    "walker_body_names": [
        "torso",
        "pelvis",
        "upper_leg_L",
        "lower_leg_L",
        "foot_L",
        "upper_leg_R",
        "lower_leg_R",
        "foot_R",
        "skull",
        "jaw",
        "scapula_L",
        "upper_arm_L",
        "lower_arm_L",
        "finger_L",
        "scapula_R",
        "upper_arm_R",
        "lower_arm_R",
        "finger_R"
    ],
    "joint_names": [
        "vertebra_1_extend",
        "hip_L_supinate",
        "hip_L_abduct",
        "hip_L_extend",
        "knee_L",
        "ankle_L",
        "toe_L",
        "hip_R_supinate",
        "hip_R_abduct",
        "hip_R_extend",
        "knee_R",
        "ankle_R",
        "toe_R",
        "vertebra_C11_extend",
        "vertebra_cervical_1_bend",
        "vertebra_axis_twist",
        "atlas",
        "mandible",
        "scapula_L_supinate",
        "scapula_L_abduct",
        "scapula_L_extend",
        "shoulder_L",
        "shoulder_sup_L",
        "elbow_L",
        "wrist_L",
        "scapula_R_supinate",
        "scapula_R_abduct",
        "scapula_R_extend",
        "shoulder_R",
        "shoulder_sup_R",
        "elbow_R",
        "wrist_R",
        "finger_R"
    ],
    "center_of_mass": "torso",
    "mjcf_path": "./assets/rodent.xml",
    "scale_factor": 0.9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 6,
    "healthy_z_range": (0.2, 1.0),
    "reset_noise_scale": 0.1,
    "clip_length": 250,
    "sub_clip_length": 10,
    "ref_traj_length": 5,
    "termination_threshold": 5,
    "body_error_multiplier": 1.0,
}

env_s = envs.get_environment('rodent_single', **env_args)

### `vmap` of the environment not available yet!

In [ ]:
key = random.PRNGKey(100)
reset_fn = jax.jit(env_s.reset)
step_fn = jax.jit(env_s.step)

n_envs = 5
_, key_envs= jax.random.split(key)
env_state = reset_fn(key_envs)

In [ ]:
env_state.info

{'cur_frame': Array(36, dtype=int32),
 'sub_clip_frame': Array(0, dtype=int32, weak_type=True),
 'termination_error': Array(0.22856492, dtype=float32),
 'traj': Array([ 0.39154962,  0.16734575,  0.11736992,  0.3782915 ,  0.15460956,
         0.11611144,  0.4061749 ,  0.12981644,  0.09224587,  0.4061749 ,
         0.12981644,  0.09224587,  0.4061749 ,  0.12981644,  0.09224587,
         0.39091292,  0.16724986,  0.1150074 ,  0.37745553,  0.15453087,
         0.11410891,  0.40458795,  0.13184093,  0.08867101,  0.40458795,
         0.13184093,  0.08867101,  0.40458795,  0.13184093,  0.08867101,
         0.39045614,  0.16842628,  0.11219793,  0.3764755 ,  0.15536475,
         0.11146784,  0.40267816,  0.1338081 ,  0.08442029,  0.40267816,
         0.1338081 ,  0.08442029,  0.40267816,  0.1338081 ,  0.08442029,
         0.3898809 ,  0.17004865,  0.10895442,  0.37572348,  0.15662089,
         0.10837589,  0.4008316 ,  0.13651828,  0.07850949,  0.4008316 ,
         0.13651828,  0.07850949,  0.

In [ ]:
step_fn = jax.jit(env_s.step)
mu = 0
sigma = 0.3
_, key = random.split(key)
action =  mu + sigma * random.normal(key, shape=(env_s.sys.nu,))
action.shape

(30,)

In [ ]:
new_state = step_fn(env_state, action)